In [1]:
# Povm

In [35]:
import pickle
import time
from typing import List
import datetime as dt
from pathlib import Path

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

from tqdm import tqdm

from quara.data_analysis import data_analysis, physicality_violation_check, report
from quara.objects.composite_system import CompositeSystem
from quara.objects.elemental_system import ElementalSystem
from quara.objects.matrix_basis import get_normalized_pauli_basis
from quara.objects.state import (
    State,
    get_x0_1q,
    get_x1_1q,
    get_y0_1q,
    get_y1_1q,
    get_z0_1q,
    get_z1_1q,
)
from quara.objects.povm import (
    Povm,get_x_measurement
)
from quara.objects.qoperation import QOperation
from quara.protocol.qtomography.standard.standard_povmt import StandardPovmt
from quara.protocol.qtomography.standard.linear_estimator import LinearEstimator
from quara.protocol.qtomography.standard.projected_linear_estimator import (
    ProjectedLinearEstimator,
)

In [36]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
start_all = time.time()

In [38]:
# setup system
e_sys = ElementalSystem(0, get_normalized_pauli_basis())
c_sys = CompositeSystem([e_sys])

# |+><+|
state_x0 = get_x0_1q(c_sys)
# |+i><+i|
state_y0 = get_y0_1q(c_sys)
# |0><0|
state_z0 = get_z0_1q(c_sys)
# |1><1|
state_z1 = get_z1_1q(c_sys)
tester_objects = [state_x0, state_y0, state_z0, state_z1]

In [39]:
# Case 1: 軸上
# a0, a1, a2, a3 = 1, 1, 0, 0
# m1 = (1 / np.sqrt(2)) * np.array([a0, a1, a2, a3])
# m2 = (1 / np.sqrt(2)) * np.array([2 - a0, -a1, -a2, -a3])

# Case 2: 軸から一番離れた表面
a0, a1, a2, a3 = 1, 1 / np.sqrt(2), 0, 1 / np.sqrt(2)
m1 = (1 / np.sqrt(2)) * np.array([a0, a1, a2, a3])
m2 = (1 / np.sqrt(2)) * np.array([2 - a0, -a1, -a2, -a3])

# Case 3: 原点
# a0, a1, a2, a3 = 1, 0, 0, 0
# m1 = (1 / np.sqrt(2)) * np.array([a0, a1, a2, a3])
# m2 = (1 / np.sqrt(2)) * np.array([2 - a0, -a1, -a2, -a3])

true_object = Povm(vecs=[m1, m2], c_sys=c_sys)

true_object.vecs

(array([0.70710678, 0.5       , 0.        , 0.5       ]),
 array([ 0.70710678, -0.5       ,  0.        , -0.5       ]))

In [42]:
num_data = [100, 1000]
n_rep = 1000
measurement_n = len(true_object.vecs)  # 測定値の数

case_name_list = [
    "LinearEstimator(True)",
    "LinearEstimator(False)",
    "ProjectedLinearEstimator(True)",
    "ProjectedLinearEstimator(False)",
]

seed = 777
qtomography_list = [
    StandardPovmt(
        tester_objects, measurement_n, on_para_eq_constraint=True, seed=seed
    ),
    StandardPovmt(
        tester_objects, measurement_n, on_para_eq_constraint=False, seed=seed
    ),
    StandardPovmt(
        tester_objects, measurement_n, on_para_eq_constraint=True, seed=seed
    ),
    StandardPovmt(
        tester_objects, measurement_n, on_para_eq_constraint=False, seed=seed
    ),
]
para_list = [True, False, True, False]

estimator_list = [
    LinearEstimator(),
    LinearEstimator(),
    ProjectedLinearEstimator(),
    ProjectedLinearEstimator(),
]

estimation_results_list = []
elapsed_times = []

for i, name in enumerate(case_name_list):
    qtomography = qtomography_list[i]
    estimator = estimator_list[i]
    
    start = time.time()
    print(f"Case {i}: {name}")
    print(f"Parametorization: {para_list[i]}")
    print(f"Type of qtomography: {qtomography.__class__.__name__}")
    print(f"Estimator: {estimator.__class__.__name__}")

    estimation_results = data_analysis.estimate(
       qtomography=qtomography,
       true_object=true_object,
       num_data=num_data,
       estimator=estimator,
       iteration=n_rep,
    )
    estimation_results_list.append(estimation_results)
    
    elapsed_time = time.time() - start
    print("elapsed_time:{0}".format(elapsed_time / 60) + "[min]\n")
    elapsed_times.append(elapsed_time)

  0%|          | 3/1000 [00:00<00:35, 28.34it/s]

Case 0: LinearEstimator(True)
Parametorization: True
Type of qtomography: StandardPovmt
Estimator: LinearEstimator


  0%|          | 0/1000 [00:00<?, ?it/s]

elapsed_time:0.8046701351801554[min]

Case 1: LinearEstimator(False)
Parametorization: False
Type of qtomography: StandardPovmt
Estimator: LinearEstimator


  0%|          | 0/1000 [00:00<?, ?it/s]/Users/tomoko/project/rcast/workspace/quara/quara/objects/povm.py:260: ComplexWarning:

Casting complex values to real discards the imaginary part



elapsed_time:0.7920575499534607[min]

Case 2: ProjectedLinearEstimator(True)
Parametorization: True
Type of qtomography: StandardPovmt
Estimator: ProjectedLinearEstimator


  0%|          | 0/1000 [00:00<?, ?it/s]

elapsed_time:5.057718932628632[min]

Case 3: ProjectedLinearEstimator(False)
Parametorization: False
Type of qtomography: StandardPovmt
Estimator: ProjectedLinearEstimator


100%|██████████| 1000/1000 [04:37<00:00,  3.60it/s]

elapsed_time:4.629573885599772[min]



In [43]:
report.export_report("sample_povmt_case02_軸から一番離れた表面_nrep=1000.pdf",
    estimation_results_list,
    case_name_list,
    qtomography_list,
    para_list,
    estimator_list,
    true_object,
    tester_objects,
    seed=seed,
    computation_time=sum(elapsed_times)
)

  0%|          | 0/1000 [00:00<?, ?it/s]

​Generating table of computation time ...
​Generating table of experimental conditions ...
Generating case list ...
​​Generating MSE of empirical distributions blocks ...


100%|██████████| 1000/1000 [00:00<00:00, 135759.96it/s]


​​Generating consictency test blocks ...
​Generating a graph for MSE ...


/Users/tomoko/project/rcast/workspace/quara/venv/lib/python3.8/site-packages/numpy/core/_methods.py:216: RuntimeWarning:

Degrees of freedom <= 0 for slice

/Users/tomoko/project/rcast/workspace/quara/venv/lib/python3.8/site-packages/numpy/core/_methods.py:209: RuntimeWarning:

invalid value encountered in double_scalars

/Users/tomoko/project/rcast/workspace/quara/quara/objects/povm.py:260: ComplexWarning:

Casting complex values to real discards the imaginary part

  0%|          | 0/1000 [00:00<?, ?it/s]

​​Generating physicality violation test blocks ...


  0%|          | 0/2 [00:00<?, ?it/s]
2it [00:00, 35.06it/s]

2it [00:00, 33.98it/s]
  0%|          | 0/2 [00:00<?, ?it/s]
0it [00:00, ?it/s]
2it [00:00,  4.44it/s]
 50%|█████     | 1/2 [00:00<00:00,  2.16it/s]
2it [00:00, 35.44it/s]
  0%|          | 0/2 [00:00<?, ?it/s]
2it [00:00, 32.54it/s]

2it [00:00, 25.05it/s]
  0%|          | 0/2 [00:00<?, ?it/s]
2it [00:00, 38.41it/s]

2it [00:00, 31.77it/s]
  0%|          | 0/2 [00:00<?, ?it/s]
2it [00:00, 45.74it/s]

2it [00:00, 28.56it/s]
  0%|          | 0/2 [00:00<?, ?it/s]
2it [00:00, 36.21it/s]

2it [00:00, 30.24it/s]
  0%|          | 0/2 [00:00<?, ?it/s]
2it [00:00, 35.69it/s]

2it [00:00, 33.21it/s]
  0%|          | 0/2 [00:00<?, ?it/s]
2it [00:00, 36.85it/s]

2it [00:00, 32.43it/s]
100%|██████████| 1000/1000 [00:00<00:00, 1353.02it/s]


Converting to PDF report ...
​Deleting temporary files ...
Completed to export pdf. (sample_povmt_case02_軸から一番離れた表面_nrep=1000.pdf)


In [44]:
# 解析解の確認
qtomography_list[0].calc_mse_linear_analytical(
    true_object, [num_data[0]] * len(tester_objects), mode="qoperation"
)

0.025000000000000015

In [47]:
# MSEの表示
fig = data_analysis.make_mses_graph_estimation_results(
    estimation_results_list,
    case_names=case_name_list,
    true_object=true_object,
    tester_objects=tester_objects,
)
print("case2: 軸から一番離れた表面")
fig.show()

case2: 軸から一番離れた表面


In [168]:
figs = physicality_violation_check.make_graphs_sum_unphysical_eigenvalues(
    estimation_results_list[3], 0
)

figs[0]

100%|██████████| 100/100 [00:00<00:00, 367.72it/s]


In [177]:
figs = physicality_violation_check.make_graphs_sum_unphysical_eigenvalues(
    estimation_results_list[3], 0
)

figs[0]

100%|██████████| 100/100 [00:00<00:00, 1086.80it/s]
